In [2]:
# importing packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
from collections import defaultdict
labelEncoder_dict = defaultdict(LabelEncoder)

# from sklearn.preprocessing import StandardScaler

In [3]:
# Loading file data
df = pd.read_csv("survey_results_public.csv")
X = pd.DataFrame()

In [4]:
X['OpenSource'] = df['OpenSource'].eq('Yes').mul(1)
X['Hobby'] = df['Hobby'].eq('Yes').mul(1)
X['Student'] = df['Student'].str.contains('Yes').mul(1)


In [5]:
def CustomLabelEncoder(data):
    le = LabelEncoder()
    labels = le.fit_transform(data)
    mappings = {index: label for index, label in enumerate(fle.classes_)}
    return le,labels


### Label Encoding

In [6]:

# Categorical Encoding Columns
CategoricalColumns = ['Country','Employment','FormalEducation','UndergradMajor','CompanySize','YearsCoding']
for col in CategoricalColumns:
    X[col] = labelEncoder_dict[col].fit_transform(df[col])

In [7]:
# AssessJob and Benefits Added
df.fillna(df.iloc[:,17:38].mean(),inplace=True)
for col in df.iloc[:,17:38].columns:
    X[col] = df[col]



In [9]:
# JobSatisfaction Mapping
SatisfactionMapping = {
    'Extremely satisfied':6,
    'Moderately dissatisfied':5,
    'Moderately satisfied':4, 
    'Neither satisfied nor dissatisfied':3,
    'Slightly satisfied':2,
    'Slightly dissatisfied':1,
    'Extremely dissatisfied':0
}

df['JobSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'JobSatisfaction':3},inplace=True)
X['JobSatisfaction'] = df['JobSatisfaction']

# CareerSatisfaction
df['CareerSatisfaction'].replace(SatisfactionMapping,inplace=True)
df.fillna({'CareerSatisfaction':3},inplace=True)
X['CareerSatisfaction'] = df['CareerSatisfaction']



In [10]:
# HackathonReasons
X['HackathonParticipated'] = df['HackathonReasons'].notna()*1


In [11]:
# ConvertedSalary
X['ConvertedSalary'] = df['ConvertedSalary']


In [12]:
def CustomOneHotEncoding(data,X):
    temp = data.str.split(';', expand=True)
    new_columns = pd.unique(temp.values.ravel())
    for col in new_columns:
        X[col] = data.str.contains(col, regex=False).fillna(False)*1

In [13]:
# LanguageWorkedWith
CustomOneHotEncoding(df['LanguageWorkedWith'],X)


CustomOneHotEncoding(df['DevType'],X)

CustomOneHotEncoding(df['DatabaseWorkedWith'],X)

CustomOneHotEncoding(df['PlatformWorkedWith'],X)

CustomOneHotEncoding(df['FrameworkWorkedWith'],X)

CustomOneHotEncoding(df['IDE'],X)

# Methodology
CustomOneHotEncoding(df['Methodology'],X)

# RaceEthnicity
CustomOneHotEncoding(df['RaceEthnicity'],X)

CustomOneHotEncoding(df['Methodology'],X)


In [15]:
# CheckInCode
CheckInCodeMapping = {
    'Multiple times per day':730, 
    'A few times per week':156, 
    'Weekly or a few times per month':52, 
    'Never':0,
    'Less than once per month':12, 
    'Once a day':365
}

X['CheckInCode'] = df['CheckInCode'].replace(CheckInCodeMapping)
X['CheckInCode'].fillna(X['CheckInCode'].mean(),inplace=True)

In [19]:
X['Age'] = df['Age']
X['MilitaryUS'] = df['MilitaryUS'].str.eq('Yes')*1
X['Dependents'] = df['Dependents'].str.eq('Yes')*1
X['Gender'] = df['Gender'].str.eq('Female')*1

(98855, 186)

In [25]:
# Onehotencoding
# df['EducationParents'].unique()
# X['Gender'] = df['Gender'].str.eq('Female')*1
# X['Gender']
df['Exercise'].unique()

array(['3 - 4 times per week', 'Daily or almost every day', nan,
       "I don't typically exercise", '1 - 2 times per week'], dtype=object)

In [ ]:
# # df['EducationTypes'].unique()

# x = .str.split(';').fillna('NA').tolist()
# devTypeSet = set()
# for devs in x:
#     try:
#         devTypeSet.update(devs)
#     except:
#         print(devs)

# list(devTypeSet)

In [ ]:
# # df['EducationTypes'].unique()

# x = df['SelfTaughtTypes'].str.split(';').fillna('NA').tolist()
# devTypeSet = set()
# for devs in x:
#     try:
#         devTypeSet.update(devs)
#     except:
#         print(devs)

# list(devTypeSet)